In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import copy
import mpi4py


from ema_workbench import (
    Model,
    Policy,
    RealParameter,
    ScalarOutcome,
    ema_logging,
    perform_experiments,
    MPIEvaluator,
    MultiprocessingEvaluator,
    save_results,
)


In [6]:
test=pd.read_csv("Filtered Policies - to be tested for robustness.csv",index_col=0)

In [7]:
test.head()

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Evacuation Costs,Expected Number of Deaths,Total Investment Costs,RfR_agg
2,0,0,0,0,0,0,1,1,0,0,...,4,0,0,0.0,1.440720e+08,61400000.0,0.0,0.0,2.054720e+08,2
4,0,0,0,0,0,0,1,1,1,0,...,4,0,0,0.0,1.422034e+08,92100000.0,0.0,0.0,2.343034e+08,3
8,0,0,0,0,0,0,1,0,0,0,...,4,0,0,0.0,1.490962e+08,30700000.0,0.0,0.0,1.797962e+08,1
16,0,0,0,0,0,0,1,0,0,0,...,3,0,0,0.0,1.410464e+08,151900000.0,0.0,0.0,2.929464e+08,2


In [9]:
test=test.drop(labels=['RfR_agg'],axis=1)

In [15]:
test.keys()

Index(['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2',
       '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2',
       '4_RfR 0', '4_RfR 1', '4_RfR 2', 'EWS_DaysToThreat',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'Expected Annual Damage', 'Dike Investment Costs',
       'RfR Investment Costs', 'Evacuation Costs', 'Expected Number of Deaths',
       'Total Investment Costs'],
      dtype='object')

In [14]:
policies=test.to_dict('index')

In [23]:
for k,v in policies.items():
    print(k)
    print(v)

2
{'0_RfR 0': 0, '0_RfR 1': 0, '0_RfR 2': 0, '1_RfR 0': 0, '1_RfR 1': 0, '1_RfR 2': 0, '2_RfR 0': 1, '2_RfR 1': 1, '2_RfR 2': 0, '3_RfR 0': 0, '3_RfR 1': 0, '3_RfR 2': 0, '4_RfR 0': 0, '4_RfR 1': 0, '4_RfR 2': 0, 'EWS_DaysToThreat': 3, 'A.1_DikeIncrease 0': 2, 'A.1_DikeIncrease 1': 0, 'A.1_DikeIncrease 2': 0, 'A.2_DikeIncrease 0': 2, 'A.2_DikeIncrease 1': 0, 'A.2_DikeIncrease 2': 0, 'A.3_DikeIncrease 0': 2, 'A.3_DikeIncrease 1': 0, 'A.3_DikeIncrease 2': 0, 'A.4_DikeIncrease 0': 3, 'A.4_DikeIncrease 1': 0, 'A.4_DikeIncrease 2': 0, 'A.5_DikeIncrease 0': 4, 'A.5_DikeIncrease 1': 0, 'A.5_DikeIncrease 2': 0, 'Expected Annual Damage': 0.0, 'Dike Investment Costs': 144071961.36120877, 'RfR Investment Costs': 61400000.0, 'Evacuation Costs': 0.0, 'Expected Number of Deaths': 0.0, 'Total Investment Costs': 205471961.3612088}
4
{'0_RfR 0': 0, '0_RfR 1': 0, '0_RfR 2': 0, '1_RfR 0': 0, '1_RfR 1': 0, '1_RfR 2': 0, '2_RfR 0': 1, '2_RfR 1': 1, '2_RfR 2': 1, '3_RfR 0': 0, '3_RfR 1': 0, '3_RfR 2': 0, '4

In [28]:
policies=[Policy(f"policy {k}", **dict(v)) for k,v in policies.items()]

In [27]:
policies_list=[]
for k,v in policies.items():
    tmp=Policy(k,**dict(v))
    policies_list.append(tmp)

    

In [30]:
policies[2]

Policy({'0_RfR 0': 0, '0_RfR 1': 0, '0_RfR 2': 0, '1_RfR 0': 0, '1_RfR 1': 0, '1_RfR 2': 0, '2_RfR 0': 1, '2_RfR 1': 0, '2_RfR 2': 0, '3_RfR 0': 0, '3_RfR 1': 0, '3_RfR 2': 0, '4_RfR 0': 0, '4_RfR 1': 0, '4_RfR 2': 0, 'EWS_DaysToThreat': 3, 'A.1_DikeIncrease 0': 2, 'A.1_DikeIncrease 1': 0, 'A.1_DikeIncrease 2': 0, 'A.2_DikeIncrease 0': 3, 'A.2_DikeIncrease 1': 0, 'A.2_DikeIncrease 2': 0, 'A.3_DikeIncrease 0': 2, 'A.3_DikeIncrease 1': 0, 'A.3_DikeIncrease 2': 0, 'A.4_DikeIncrease 0': 3, 'A.4_DikeIncrease 1': 0, 'A.4_DikeIncrease 2': 0, 'A.5_DikeIncrease 0': 4, 'A.5_DikeIncrease 1': 0, 'A.5_DikeIncrease 2': 0, 'Expected Annual Damage': 0.0, 'Dike Investment Costs': 149096245.25281107, 'RfR Investment Costs': 30700000.0, 'Evacuation Costs': 0.0, 'Expected Number of Deaths': 0.0, 'Total Investment Costs': 179796245.25281107})